In [2]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

In [5]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

In [ ]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

  def __init__(self, url):
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    self.title = soup.title.string if soup.title else "No title found"
    for irrelevant in soup.body(["script", "style", "img", "input"]):
      irrelevant.decompose()
    self.text = soup.body.get_text(separator="\n", strip=True)

ed = Website("https://edwarddonner.com")
# print(ed.title)
# print(ed.text)

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

def user_prompt_for(website):
  user_prompt = f"You are looking at a website titled {website.title}"
  user_prompt += "\nThe contents of this website is as follows; \
  please provide a short summary of this website in markdown. \
  If it includes news or announcements, then summarize these too.\n\n"
  user_prompt += website.text
  return user_prompt

user_prompt = user_prompt_for(ed)

openai = OpenAI(
  api_key=api_key,
  base_url="https://api.groq.com/openai/v1"
)
response = openai.chat.completions.create(model="llama-3.1-8b-instant", messages=[
  { "role": "system", "content": system_prompt },
  { "role": "user", "content": user_prompt }
])
print(response.choices[0].message.content)